In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
def coord(image):
    image = cv2.imread(image,cv2.IMREAD_UNCHANGED)
    # Ensure image has an alpha channel
    if image.shape[2] < 4:
        raise ValueError("Image does not have an alpha channel")

    # Extract the alpha channel
    alpha = image[:, :, 3]

    # Find all non-transparent pixels
    coords = cv2.findNonZero(alpha)  # Returns coordinates of non-zero (non-transparent) pixels

    # Get bounding box for non-transparent region
    x, y, w, h = cv2.boundingRect(coords)

    return x,y,w,h

In [5]:
troops = ["valkyrie"]
data = pd.read_csv("image_centers/troops_center.csv")

for troop in troops:
    print(troop)
    image = f'images/troops/Blue/{troop}_run_n_6.png'
    x,y,w,h = coord(image)
    new_row = pd.DataFrame({'troop': [troop], 'x': [x], "y" : [y], "w": [w], "h" : [h]})
    data = pd.concat([data, new_row], ignore_index=True)
data.to_csv("image_centers/troops_center.csv",index=False)

valkyrie


In [1]:
import os
from PIL import Image, ImageFilter
def preprocess(input_path,output_path):
    image = Image.open(input_path).convert("RGBA")  # Preserve transparency

    bbox = image.getbbox()
    if not bbox:
        raise ValueError("No bounding box found. Ensure the image is not fully transparent.")

    #get the last number of the bbox
    image_last_co= bbox[-1]

    # Create a black shadow based on the entire image (no cropping)
    shadow = Image.new("RGBA", image.size, (0, 0, 0, 120))  # Fully black shadow
    shadow_mask = image.split()[3]  # Use the alpha channel of the original image

    # Stretch the shadow downward (project it on the ground)
    shadow_width = image.width
    shadow_height = int(image.height*0.3)   # Adjust how far the shadow extends
    shadow = Image.new("RGBA", (shadow_width, shadow_height), (0, 0, 0, 120))  # Black semi-transparent shadow
    shadow_mask = image.split()[3].resize((shadow_width, shadow_height))  # Resize mask to match shadow

    # Flip the shadow vertically
    shadow = shadow.transpose(Image.FLIP_TOP_BOTTOM)
    shadow_mask = shadow_mask.transpose(Image.FLIP_TOP_BOTTOM)

    # Apply blur to soften the shadow
    shadow = shadow.filter(ImageFilter.GaussianBlur(10))
    # Keep the new image size exactly the same as the original
    new_image = Image.new("RGBA", (image.width,int(image.height*1.3)), (255, 255, 255, 0))  # No extra height added
    shadow_bbox = shadow.getbbox()


    # get the last coordinate of the shadow
    shadow_last_co = shadow_bbox[-1]
    diff=shadow_last_co-image_last_co


    # Positioning
    image_x = 0
    image_y = 0  # Keep the original image in its position
    shadow_x = 0  # Align shadow with image
    shadow_y = int(image.height*0.2)-diff# Slight overlap to blend smoothly

    # Paste the shadow first
    new_image.paste(shadow, (shadow_x, shadow_y), mask=shadow_mask)

    new_image.save(output_path)

def shadow():

    # Define input and output directories
    input_ = "/Users/tusharsingharoy/coderoyale/data/images/troops/"
    output = "/Users/tusharsingharoy/coderoyale/data/images/troops/"
    input_dir2 = '/Users/tusharsingharoy/coderoyale/data/images/tower/Blue/BlueTower.png'
    output_dir2 = '/Users/tusharsingharoy/coderoyale/data/images/tower/Blue/BlueTower_shadow.png'
    input_dir3 = '/Users/tusharsingharoy/coderoyale/data/images/tower/Red/RedTower.png'
    output_dir3 = '/Users/tusharsingharoy/coderoyale/data/images/tower/Red/RedTower_shadow.png'
    
    for dire in ["Red","Blue"]:
        input_dir = input_ + dire
        output_dir = output + dire + "_shadows"
        # Loop through all files in the input directory
        for filename in os.listdir(input_dir):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            preprocess(input_path,output_path)
    preprocess(input_dir2,output_dir2)
    preprocess(input_dir3,output_dir3)

            
shadow()
